<h2>Initializing</h2>
<p>Import Libraries, add variables for attribute strings (to save '') and load data</p>

In [1]:
import pandas as pd
import numpy as np
from scipy.stats.stats import pearsonr
from sklearn.cluster import KMeans
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix 
from sklearn import tree
from sklearn.metrics import classification_report
from scipy import interp
from sklearn.metrics import accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_validate
import math

fixed_acidity = 'fixed acidity'
volatile_acidity = 'volatile acidity'
citric_acid = 'citric acid'
residual_sugar = 'residual sugar'
chlorides = 'chlorides'
free_sulfur_dioxide = 'free sulfur dioxide'
total_sulfur_dioxide = 'total sulfur dioxide'
density = 'density'
ph = 'pH'
sulphates = 'sulphates'
alcohol = 'alcohol'
quality = 'quality'
qclass = 'qclass'
ratio_to_fixed = 'ratio_to_fixed'
ratio_to_volatile = 'ratio_to_volatile'
ph_acidity = 'ph_acidity'
ratio_to_ph = 'ratio_to_ph'

In [2]:
def confusion_matrix_report(y_true, y_pred):
    cm, labels = confusion_matrix(y_true, y_pred), unique_labels(y_true, y_pred)
    column_width = max([len(str(x)) for x in labels] + [5])  # 5 is value length
    report = " " * column_width + " " + "{:_^{}}".format("Prediction", column_width * len(labels))+ "\n"
    report += " " * column_width + " ".join(["{:>{}}".format(label, column_width) for label in labels]) + "\n"
    for i, label1 in enumerate(labels):
        report += "{:>{}}".format(label1, column_width) + " ".join(["{:{}d}".format(cm[i, j], column_width) for j in range(len(labels))]) + "\n"
    return report
def root_mean_squared_error(y_true, y_pred):
        return math.sqrt(np.mean((y_pred - y_true)**2))

<p>Import red and white dataset.<br/>
Split into input matrix (independent vars) and output vector (target vars).<br/>
</p>

<p>sort for kNN (https://scikit-learn.org/stable/modules/neighbors.html#unsupervised-nearest-neighbors) </p>

In [3]:
white = pd.read_csv("data/winequality-white1.csv")
red = pd.read_csv("data/winequality-red1.csv")
white.sort_values(by='quality',inplace=True)
red.sort_values(by='quality',inplace=True)
white_target = white[quality]
red_target = red[quality]
white_input = white.drop(quality,axis=1)
red_input = red.drop(quality,axis=1)

Add ratio of acidity to sugar as attribute.

In [4]:
#white[ph_acidity] = 7 - white[ph]
#red[ph_acidity] = 7 - red[ph]
#white_input[ratio_to_fixed]=white[residual_sugar]/white[fixed_acidity]
#white_input[ratio_to_volatile]=white[residual_sugar]/white[volatile_acidity]
#white_input[ratio_to_ph]=white[residual_sugar]/white[ph_acidity]
#red_input[ratio_to_fixed]=red[residual_sugar]/red[fixed_acidity]
#red_input[ratio_to_volatile]=red[residual_sugar]/red[volatile_acidity]
#red_input[ratio_to_ph]=red[residual_sugar]/red[ph_acidity]

<p>using log. (and filtering based on results of naive bayes)</p>

In [5]:
log_white_norm=np.log(white_input+1)
log_red_norm=np.log(red_input+1)
f_log_white=log_white_norm.drop([fixed_acidity,sulphates,chlorides,citric_acid,ph,density,total_sulfur_dioxide,residual_sugar],axis=1)
f_log_red=log_red_norm.drop([free_sulfur_dioxide,ph,residual_sugar,chlorides,citric_acid,fixed_acidity,density],axis=1)

<p>Normalize using MinMax Normalizer. (and filtering based on results of naive bayes)</p>

In [6]:
mm_white_norm=(white_input-white_input.min())/(white_input.max()-white_input.min())
mm_red_norm=(red_input-red_input.min())/(red_input.max()-red_input.min())
f_mm_white = mm_white_norm.drop([fixed_acidity,sulphates,chlorides,citric_acid,ph,density,total_sulfur_dioxide,residual_sugar],axis=1)
f_mm_red = mm_red_norm.drop([free_sulfur_dioxide,ph,residual_sugar,chlorides,citric_acid,fixed_acidity,density],axis=1)

<p>Normalize using Std. Normalizer. (and filtering based on results of naive bayes)</p>

In [7]:
white_norm=(white_input-white_input.mean())/(white_input.std())
red_norm=(red_input-red_input.mean())/(red_input.std())
f_white_norm=white_norm.drop([fixed_acidity,sulphates,chlorides,citric_acid,ph,density,total_sulfur_dioxide,residual_sugar],axis=1)
f_red_norm=red_norm.drop([free_sulfur_dioxide,ph,residual_sugar,chlorides,citric_acid,fixed_acidity,density],axis=1)

<p>Correlation Matrices</p>

In [8]:
#wcorr = white.corr()
#wcorr.style.background_gradient().set_precision(3)

In [9]:
#rcorr = red.corr()
#rcorr.style.background_gradient().set_precision(3)

<h2>Assigning classes</h2>
<p>
    Assign classes based on quality. Less than 6; 6; better than 6. Because it makes sense considering the distribution of ratings.
</p>

In [10]:
#Assign classes for white
conditions = [(white['quality'] < 6),
             (white['quality'] > 6)]
choices = [1,3]
white['classnum'] = np.select(conditions, choices, default = 2)
white_classnum = white['classnum']
white.drop('classnum',axis=1,inplace=True)
#choices = ['2_low', '1_high']
#white['class'] = np.select(conditions, choices, default = '3_medium')
#choices = [1,3]
#white['classnum'] = np.select(conditions, choices, default = 2)
#white_classnum = white['classnum']
#white.drop('classnum',axis=1,inplace=True)
#Assign classes for red
conditions = [(red['quality'] < 6),
              (red['quality'] > 6)]
#choices = ['3_low', '1_high']
#red['class'] = np.select(conditions, choices, default = '2_medium')
#choices = [1,3]
red['classnum'] = np.select(conditions, choices, default = 2)
red_classnum = red['classnum']
red.drop('classnum',axis=1,inplace=True)

#print('class distribution fot white:')
#print(white['class'].value_counts())
#print('\n class distribution for red:')
#print(red['class'].value_counts())

#white_norm['class']=white['class']
#red_norm['class']=red['class']

white_targetclass = white_classnum
red_targetclass = red_classnum
print(white_targetclass.value_counts())
print(red_targetclass.value_counts())
#white_targetclass = white['class']
#red_targetclass = red['class']

2    2198
1    1640
3    1060
Name: classnum, dtype: int64
1    744
2    638
3    217
Name: classnum, dtype: int64


In [11]:
white_norm_input = white_norm
red_norm_input = red_norm
print(white_norm_input.head())
print(red_norm_input.head())

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
3307       3.016125         -0.379397    -0.365159        0.415726  -0.401518   
445        0.290581          0.414297    -0.117266        0.908622  -0.355747   
2050       5.860171         -0.478608     0.378521        0.928338  -0.538831   
3810      -0.064924         -0.180973     0.047996        1.716973   0.651211   
3409      -0.775936         -0.478608     0.130627       -1.122112   0.239274   

      free sulfur dioxide  total sulfur dioxide   density        pH  \
3307             5.214982              1.638647 -0.025871 -1.909043   
445             -1.135293             -1.702681 -0.109457  0.342604   
2050            -1.194092             -0.361444  1.896623 -1.710368   
3810             0.393477              0.556245  1.010604  0.342604   
3409            -0.664902             -0.643809 -0.811585  1.203527   

      sulphates   alcohol  
3307  -0.962507  0.394706  
445   -0.787262  0.801005  
20

In [12]:
#white_norm.loc[white_norm['class']=='3_high'].describe()
#white_norm.loc[white_norm['class']=='2_medium'].describe()
#white_norm.loc[white_norm['class']=='1_low'].describe()
white_norm_filtered = white_norm_input.drop([fixed_acidity,sulphates,chlorides,citric_acid,ph,density,total_sulfur_dioxide,residual_sugar],axis=1)
white_norm_filtered.describe()

,volatile acidity,free sulfur dioxide,alcohol
count,4.898000e+03,4.898000e+03,4.898000e+03
mean,-1.998539e-14,-6.380722e-17,-4.325046e-14
std,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.966784e+00,-1.958477e+00,-2.043089e+00
25%,-6.770318e-01,-7.237012e-01,-8.241915e-01
50%,-1.809733e-01,-7.691388e-02,-9.285319e-02
75%,4.142970e-01,6.286722e-01,7.197450e-01
max,8.152811e+00,1.491679e+01,2.995020e+00


In [13]:
#red_norm.loc[red_norm['class']=='3_high'].describe()
#red_norm.loc[red_norm['class']=='2_medium'].describe()
#red_norm.loc[red_norm['class']=='1_low'].describe()
red_norm_filtered = red_norm_input.drop([free_sulfur_dioxide,ph,residual_sugar,chlorides,citric_acid,fixed_acidity,density],axis=1)
red_norm_filtered.describe()

,volatile acidity,total sulfur dioxide,sulphates,alcohol
count,1.599000e+03,1.599000e+03,1.599000e+03,1.599000e+03
mean,8.477896e-15,2.365560e-16,6.625737e-15,2.204136e-14
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-2.277567e+00,-1.230199e+00,-1.935902e+00,-1.898325e+00
25%,-7.696903e-01,-7.438076e-01,-6.380200e-01,-8.661079e-01
50%,-4.367545e-02,-2.574163e-01,-2.250577e-01,-2.092427e-01
75%,6.264921e-01,4.721707e-01,4.238832e-01,6.352984e-01
max,5.876138e+00,7.372847e+00,7.916200e+00,4.201138e+00


<h2>k-NN classification<h2>

In [14]:
white_norm_filtered.describe()

,volatile acidity,free sulfur dioxide,alcohol
count,4.898000e+03,4.898000e+03,4.898000e+03
mean,-1.998539e-14,-6.380722e-17,-4.325046e-14
std,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.966784e+00,-1.958477e+00,-2.043089e+00
25%,-6.770318e-01,-7.237012e-01,-8.241915e-01
50%,-1.809733e-01,-7.691388e-02,-9.285319e-02
75%,4.142970e-01,6.286722e-01,7.197450e-01
max,8.152811e+00,1.491679e+01,2.995020e+00


<h3>white: knn normalized and filtered based on Naive Bayes results</h3>
RMSE/P/R/F1
<p>White,STD Normed, filteredNB:0.68/0.66/0.66/0.66 at k=1, decreasing after<br>
White log normed, filteredNB: 0.69/0.64/0.64/0.64 at k=1, decreasing after<br/>
White MM normed, filteredNB: 0.68/0.65/0.65/0.65 at k=1, decreasing after</p>

In [15]:
def run_knn_report(inputs, targets):
    print("_______________________________________________________________________________________________")
    print("KNN, no weights")
    for n_neighbour in range(1,21):
        print(str(n_neighbour) + " neighbours:")
        knn_estimator = KNeighborsClassifier(n_neighbour)
        predicted = cross_val_predict(knn_estimator,inputs,targets,cv=cv)
        print(confusion_matrix_report(targets,predicted))
        print(classification_report(targets,predicted))
        try:
            print("RMSE:")
            print(root_mean_squared_error(targets,predicted))
        except(Error):
            pass
    
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
#cv = KFold(n_splits=3)
print("#____________________white STD normed filtered____________________#")
run_knn_report(f_white_norm.values, white_targetclass)

#____________________white STD normed filtered____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1129   416    95
    2  428  1432   338
    3   86   332   642

             precision    recall  f1-score   support

          1       0.69      0.69      0.69      1640
          2       0.66      0.65      0.65      2198
          3       0.60      0.61      0.60      1060

avg / total       0.65      0.65      0.65      4898

RMSE:
0.6759594605620941
2 neighbours:
      __Prediction___
         1     2     3
    1 1306   296    38
    2  796  1243   159
    3  193   492   375

             precision    recall  f1-score   support

          1       0.57      0.80      0.66      1640
          2       0.61      0.57      0.59      2198
          3       0.66      0.35      0.46      1060

avg / total       0.61      0.60      0.59      4898

R

             precision    recall  f1-score   support

          1       0.63      0.63      0.63      1640
          2       0.54      0.63      0.58      2198
          3       0.55      0.36      0.44      1060

avg / total       0.57      0.57      0.57      4898

RMSE:
0.7013083497209496


In [16]:
print("#____________________white log normed filtered____________________#")
run_knn_report(f_log_white.values, white_targetclass)

#____________________white log normed filtered____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1090   455    95
    2  459  1372   367
    3  102   330   628

             precision    recall  f1-score   support

          1       0.66      0.66      0.66      1640
          2       0.64      0.62      0.63      2198
          3       0.58      0.59      0.58      1060

avg / total       0.63      0.63      0.63      4898

RMSE:
0.6998512354317896
2 neighbours:
      __Prediction___
         1     2     3
    1 1278   325    37
    2  807  1229   162
    3  203   507   350

             precision    recall  f1-score   support

          1       0.56      0.78      0.65      1640
          2       0.60      0.56      0.58      2198
          3       0.64      0.33      0.44      1060

avg / total       0.59      0.58      0.57      4898

R

In [17]:
print("#____________________white MinMax normed filtered____________________#")
run_knn_report(f_mm_white.values, white_targetclass)

#____________________white MinMax normed filtered____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1131   414    95
    2  441  1419   338
    3   82   325   653

             precision    recall  f1-score   support

          1       0.68      0.69      0.69      1640
          2       0.66      0.65      0.65      2198
          3       0.60      0.62      0.61      1060

avg / total       0.65      0.65      0.65      4898

RMSE:
0.6741448010182188
2 neighbours:
      __Prediction___
         1     2     3
    1 1285   319    36
    2  796  1242   160
    3  183   503   374

             precision    recall  f1-score   support

          1       0.57      0.78      0.66      1640
          2       0.60      0.57      0.58      2198
          3       0.66      0.35      0.46      1060

avg / total       0.60      0.59      0.58      4898

      __Prediction___
         1     2     3
    1 1018   577    45
    2  521  1386   291
    3   64   595   401

             precision    recall  f1-score   support

          1       0.64      0.62      0.63      1640
          2       0.54      0.63      0.58      2198
          3       0.54      0.38      0.45      1060

avg / total       0.57      0.57      0.57      4898

RMSE:
0.7029076866890369


<h3><i>red: </i>knn normalized and filtered based on Naive Bayes results</h3>
<p>
    STD,NBF: 0.6/0.7/0.7/0.7 at k=1, decreasing after<br>
    MM,NBF: 0.6/0.7/0.7/0.7 at k=1, decreasing after<br>
    log NBF: 0.59/0.69/0.69/0.69 at k=1, decreasing after<br>
</p>

In [18]:
print("#____________________red STD normed____________________#")
run_knn_report(f_red_norm.values, red_targetclass)

#____________________red STD normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  576   155    13
    2  150   407    81
    3   17    70   130

             precision    recall  f1-score   support

          1       0.78      0.77      0.77       744
          2       0.64      0.64      0.64       638
          3       0.58      0.60      0.59       217

avg / total       0.70      0.70      0.70      1599

RMSE:
0.6001875879364265
2 neighbours:
      __Prediction___
         1     2     3
    1  649    88     7
    2  268   333    37
    3   39   118    60

             precision    recall  f1-score   support

          1       0.68      0.87      0.76       744
          2       0.62      0.52      0.57       638
          3       0.58      0.28      0.37       217

avg / total       0.64      0.65      0.63      1599

RMSE:
0.6592

      __Prediction___
         1     2     3
    1  581   157     6
    2  213   381    44
    3   21   118    78

             precision    recall  f1-score   support

          1       0.71      0.78      0.75       744
          2       0.58      0.60      0.59       638
          3       0.61      0.36      0.45       217

avg / total       0.65      0.65      0.64      1599

RMSE:
0.6326532670805687


In [19]:
print("#____________________red MinMax normed____________________#")
run_knn_report(f_mm_red.values, red_targetclass)

#____________________red MinMax normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  577   152    15
    2  139   420    79
    3   16    68   133

             precision    recall  f1-score   support

          1       0.79      0.78      0.78       744
          2       0.66      0.66      0.66       638
          3       0.59      0.61      0.60       217

avg / total       0.71      0.71      0.71      1599

RMSE:
0.5928487737550271
2 neighbours:
      __Prediction___
         1     2     3
    1  641    99     4
    2  268   334    36
    3   37   122    58

             precision    recall  f1-score   support

          1       0.68      0.86      0.76       744
          2       0.60      0.52      0.56       638
          3       0.59      0.27      0.37       217

avg / total       0.64      0.65      0.63      1599

RMSE:
0.6

      __Prediction___
         1     2     3
    1  573   167     4
    2  219   369    50
    3   20   112    85

             precision    recall  f1-score   support

          1       0.71      0.77      0.74       744
          2       0.57      0.58      0.57       638
          3       0.61      0.39      0.48       217

avg / total       0.64      0.64      0.64      1599

RMSE:
0.6346272290288927


In [20]:
print("#____________________red log normed____________________#")
run_knn_report(f_log_red.values, red_targetclass)

#____________________red log normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  563   162    19
    2  159   395    84
    3   18    77   122

             precision    recall  f1-score   support

          1       0.76      0.76      0.76       744
          2       0.62      0.62      0.62       638
          3       0.54      0.56      0.55       217

avg / total       0.68      0.68      0.68      1599

RMSE:
0.6276912040603917
2 neighbours:
      __Prediction___
         1     2     3
    1  643    94     7
    2  294   311    33
    3   37   116    64

             precision    recall  f1-score   support

          1       0.66      0.86      0.75       744
          2       0.60      0.49      0.54       638
          3       0.62      0.29      0.40       217

avg / total       0.63      0.64      0.62      1599

RMSE:
0.6677

             precision    recall  f1-score   support

          1       0.68      0.77      0.72       744
          2       0.54      0.56      0.55       638
          3       0.52      0.25      0.34       217

avg / total       0.61      0.61      0.60      1599

RMSE:
0.6658844692053987


<h3>*white: normalized unfiltered</h3>
<p>
    STD: 0.62/0.7/0.7/0.7 at k=1, decreasing after<br>
    MM: 0.62/0.7/0.7/0.7 at k=1, decreasing after<br>
    log: 0.65/0.67/0.67/0.67 at k=1, decreasing after<br>
</p>

In [21]:
print("#____________________white STD normed____________________#")
run_knn_report(white_norm.values, white_targetclass)

#____________________white STD normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1153   420    67
    2  372  1543   283
    3   57   277   726

             precision    recall  f1-score   support

          1       0.73      0.70      0.72      1640
          2       0.69      0.70      0.70      2198
          3       0.67      0.68      0.68      1060

avg / total       0.70      0.70      0.70      4898

RMSE:
0.6142449477688314
2 neighbours:
      __Prediction___
         1     2     3
    1 1324   292    24
    2  753  1318   127
    3  137   450   473

             precision    recall  f1-score   support

          1       0.60      0.81      0.69      1640
          2       0.64      0.60      0.62      2198
          3       0.76      0.45      0.56      1060

avg / total       0.65      0.64      0.63      4898

RMSE:
0.68

      __Prediction___
         1     2     3
    1 1054   529    57
    2  510  1401   287
    3   63   499   498

             precision    recall  f1-score   support

          1       0.65      0.64      0.65      1640
          2       0.58      0.64      0.61      2198
          3       0.59      0.47      0.52      1060

avg / total       0.60      0.60      0.60      4898

RMSE:
0.6860030940148582


In [22]:
print("#____________________white MinMax normed____________________#")
run_knn_report(mm_white_norm.values, white_targetclass)

#____________________white MinMax normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1153   412    75
    2  380  1537   281
    3   57   288   715

             precision    recall  f1-score   support

          1       0.73      0.70      0.71      1640
          2       0.69      0.70      0.69      2198
          3       0.67      0.67      0.67      1060

avg / total       0.70      0.70      0.70      4898

RMSE:
0.6210214323487594
2 neighbours:
      __Prediction___
         1     2     3
    1 1321   283    36
    2  762  1308   128
    3  126   456   478

             precision    recall  f1-score   support

          1       0.60      0.81      0.69      1640
          2       0.64      0.60      0.62      2198
          3       0.74      0.45      0.56      1060

avg / total       0.65      0.63      0.63      4898

RMSE:
0

      __Prediction___
         1     2     3
    1 1012   557    71
    2  540  1347   311
    3   47   519   494

             precision    recall  f1-score   support

          1       0.63      0.62      0.62      1640
          2       0.56      0.61      0.58      2198
          3       0.56      0.47      0.51      1060

avg / total       0.58      0.58      0.58      4898

RMSE:
0.6998512354317896


In [23]:
print("#____________________white log normed____________________#")
run_knn_report(log_white_norm.values, white_targetclass)

#____________________white log normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1151   402    87
    2  420  1485   293
    3   69   299   692

             precision    recall  f1-score   support

          1       0.70      0.70      0.70      1640
          2       0.68      0.68      0.68      2198
          3       0.65      0.65      0.65      1060

avg / total       0.68      0.68      0.68      4898

RMSE:
0.6450489898178324
2 neighbours:
      __Prediction___
         1     2     3
    1 1285   318    37
    2  762  1295   141
    3  155   467   438

             precision    recall  f1-score   support

          1       0.58      0.78      0.67      1640
          2       0.62      0.59      0.61      2198
          3       0.71      0.41      0.52      1060

avg / total       0.63      0.62      0.61      4898

RMSE:
0.70

      __Prediction___
         1     2     3
    1  958   614    68
    2  524  1405   269
    3   79   569   412

             precision    recall  f1-score   support

          1       0.61      0.58      0.60      1640
          2       0.54      0.64      0.59      2198
          3       0.55      0.39      0.46      1060

avg / total       0.57      0.57      0.56      4898

RMSE:
0.7235184662526416


<h3><i>*red: </i> normalized unfiltered</h3>
<p>
    STD: 0.63/0.69/0.69/0.69. k=1 more=bad<br>
    MM: 0.63/0.69/0.69/0.69. k=1 more=bad<br>
    log: 0.63/0.68/0.68/0.68. k=1 more=bad<br>
</p>

In [24]:
print("#____________________red STD normed____________________#")
run_knn_report(red_norm.values, red_targetclass)

#____________________red STD normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  563   157    24
    2  162   404    72
    3   24    64   129

             precision    recall  f1-score   support

          1       0.75      0.76      0.75       744
          2       0.65      0.63      0.64       638
          3       0.57      0.59      0.58       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.6361036805684829
2 neighbours:
      __Prediction___
         1     2     3
    1  625   109    10
    2  289   316    33
    3   39   101    77

             precision    recall  f1-score   support

          1       0.66      0.84      0.74       744
          2       0.60      0.50      0.54       638
          3       0.64      0.35      0.46       217

avg / total       0.63      0.64      0.62      1599

RMSE:
0.6747

      __Prediction___
         1     2     3
    1  560   168    16
    2  214   358    66
    3   23   109    85

             precision    recall  f1-score   support

          1       0.70      0.75      0.73       744
          2       0.56      0.56      0.56       638
          3       0.51      0.39      0.44       217

avg / total       0.62      0.63      0.62      1599

RMSE:
0.6677602038203002


In [25]:
print("#____________________red MinMax normed____________________#")
run_knn_report(mm_red_norm.values, red_targetclass)

#____________________red MinMax normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  555   166    23
    2  161   414    63
    3   24    64   129

             precision    recall  f1-score   support

          1       0.75      0.75      0.75       744
          2       0.64      0.65      0.65       638
          3       0.60      0.59      0.60       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.6336410167329005
2 neighbours:
      __Prediction___
         1     2     3
    1  618   122     4
    2  288   324    26
    3   46    93    78

             precision    recall  f1-score   support

          1       0.65      0.83      0.73       744
          2       0.60      0.51      0.55       638
          3       0.72      0.36      0.48       217

avg / total       0.64      0.64      0.62      1599

RMSE:
0.6

      __Prediction___
         1     2     3
    1  576   155    13
    2  230   351    57
    3   21   115    81

             precision    recall  f1-score   support

          1       0.70      0.77      0.73       744
          2       0.57      0.55      0.56       638
          3       0.54      0.37      0.44       217

avg / total       0.62      0.63      0.62      1599

RMSE:
0.6583280887371149


In [26]:
print("#____________________red log normed____________________#")
run_knn_report(log_red_norm.values, red_targetclass)

#____________________red log normed____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  555   162    27
    2  171   392    75
    3   21    67   129

             precision    recall  f1-score   support

          1       0.74      0.75      0.74       744
          2       0.63      0.61      0.62       638
          3       0.56      0.59      0.58       217

avg / total       0.67      0.67      0.67      1599

RMSE:
0.6458604414412116
2 neighbours:
      __Prediction___
         1     2     3
    1  634    99    11
    2  321   285    32
    3   53    96    68

             precision    recall  f1-score   support

          1       0.63      0.85      0.72       744
          2       0.59      0.45      0.51       638
          3       0.61      0.31      0.41       217

avg / total       0.61      0.62      0.60      1599

RMSE:
0.7090

<h3> white: unfiltered, unnormalized <h3>

In [27]:
print("#____________________white_____________________#")
run_knn_report(white_input.values,white_targetclass)

#____________________white_____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1044   451   145
    2  430  1436   332
    3  113   329   618

             precision    recall  f1-score   support

          1       0.66      0.64      0.65      1640
          2       0.65      0.65      0.65      2198
          3       0.56      0.58      0.57      1060

avg / total       0.63      0.63      0.63      4898

RMSE:
0.7249280106751942
2 neighbours:
      __Prediction___
         1     2     3
    1 1220   361    59
    2  868  1187   143
    3  252   463   345

             precision    recall  f1-score   support

          1       0.52      0.74      0.61      1640
          2       0.59      0.54      0.56      2198
          3       0.63      0.33      0.43      1060

avg / total       0.58      0.56      0.55      4898

RMSE:
0.792858075656

<p>
    unnormed white: 0.71/0.64/0.64/0.64, k=1 more bad
</p>

<h3>red, unnormalized, unselected</h3>

In [28]:
print("#____________________red_____________________#")
run_knn_report(white_input.values,white_targetclass)

#____________________red_____________________#
_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1044   451   145
    2  430  1436   332
    3  113   329   618

             precision    recall  f1-score   support

          1       0.66      0.64      0.65      1640
          2       0.65      0.65      0.65      2198
          3       0.56      0.58      0.57      1060

avg / total       0.63      0.63      0.63      4898

RMSE:
0.7249280106751942
2 neighbours:
      __Prediction___
         1     2     3
    1 1220   361    59
    2  868  1187   143
    3  252   463   345

             precision    recall  f1-score   support

          1       0.52      0.74      0.61      1640
          2       0.59      0.54      0.56      2198
          3       0.63      0.33      0.43      1060

avg / total       0.58      0.56      0.55      4898

RMSE:
0.79285807565662

      __Prediction___
         1     2     3
    1  847   709    84
    2  628  1335   235
    3  181   612   267

             precision    recall  f1-score   support

          1       0.51      0.52      0.51      1640
          2       0.50      0.61      0.55      2198
          3       0.46      0.25      0.32      1060

avg / total       0.50      0.50      0.49      4898

RMSE:
0.8138250103106349


0.72 0.64      0.64      0.64


In [29]:
def run_knc_report(inputs, targets):
    knc_estimator = NearestCentroid()
    predicted = cross_val_predict(knc_estimator,inputs,targets,cv=cv)
    print(confusion_matrix_report(targets,predicted))
    print(classification_report(targets,predicted))
    try:
        print("RMSE:")
        print(root_mean_squared_error(targets,predicted))
    except(Error):
        pass

In [30]:
print("#____________________white STD normed____________________#")
run_knc_report(white_norm.values, white_targetclass)

#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1078   272   290
    2  783   537   878
    3  138   181   741

             precision    recall  f1-score   support

          1       0.54      0.66      0.59      1640
          2       0.54      0.24      0.34      2198
          3       0.39      0.70      0.50      1060

avg / total       0.51      0.48      0.46      4898

RMSE:
0.8838185092014216


In [31]:
print("#____________________white MinMax normed____________________#")
run_knc_report(mm_white_norm.values, white_targetclass)

#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1142   298   200
    2  847   556   795
    3  145   196   719

             precision    recall  f1-score   support

          1       0.54      0.70      0.61      1640
          2       0.53      0.25      0.34      2198
          3       0.42      0.68      0.52      1060

avg / total       0.51      0.49      0.47      4898

RMSE:
0.8472567603545674


In [32]:

print("#____________________white log normed____________________#")
run_knc_report(log_white_norm.values, white_targetclass)

#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1  921    97   622
    2  974   246   978
    3  301   147   612

             precision    recall  f1-score   support

          1       0.42      0.56      0.48      1640
          2       0.50      0.11      0.18      2198
          3       0.28      0.58      0.37      1060

avg / total       0.43      0.36      0.32      4898

RMSE:
1.0964138432357722


In [33]:
print("#____________________red STD normed____________________#")
run_knc_report(red_norm.values, red_targetclass)

#____________________red STD normed____________________#
      __Prediction___
         1     2     3
    1  541   139    64
    2  219   223   196
    3   11    42   164

             precision    recall  f1-score   support

          1       0.70      0.73      0.71       744
          2       0.55      0.35      0.43       638
          3       0.39      0.76      0.51       217

avg / total       0.60      0.58      0.57      1599

RMSE:
0.7485654406174548


In [34]:
print("#____________________red MinMax normed____________________#")
run_knc_report(mm_red_norm.values, red_targetclass)

#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  561   109    74
    2  241   200   197
    3   11    44   162

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.57      0.31      0.40       638
          3       0.37      0.75      0.50       217

avg / total       0.60      0.58      0.56      1599

RMSE:
0.763045804203175


In [35]:
print("#____________________red log normed____________________#")
run_knc_report(log_red_norm.values, red_targetclass)

#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  427    84   233
    2  290    96   252
    3   66    25   126

             precision    recall  f1-score   support

          1       0.55      0.57      0.56       744
          2       0.47      0.15      0.23       638
          3       0.21      0.58      0.30       217

avg / total       0.47      0.41      0.39      1599

RMSE:
1.0747543605795422


In [36]:
def run_kncs_report(inputs, targets,thresh=0):
    knc_estimator = NearestCentroid(shrink_threshold=thresh)
    predicted = cross_val_predict(knc_estimator,inputs,targets,cv=cv)
    print(confusion_matrix_report(targets,predicted))
    print(classification_report(targets,predicted))
    try:
        print("RMSE:")
        print(root_mean_squared_error(targets,predicted))
    except(Error):
        pass

In [37]:
for t in range (0,10000,100):
    t = t/1000
    print(t)
    print("#____________________white STD normed____________________#")
    run_kncs_report(white_norm.values, white_targetclass,t)
    print("#____________________white MinMax normed____________________#")
    run_kncs_report(mm_white_norm.values, white_targetclass,t)
    print("#____________________white log normed____________________#")
    run_kncs_report(log_white_norm.values, white_targetclass,t)

0.0
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1078   272   290
    2  783   537   878
    3  138   181   741

             precision    recall  f1-score   support

          1       0.54      0.66      0.59      1640
          2       0.54      0.24      0.34      2198
          3       0.39      0.70      0.50      1060

avg / total       0.51      0.48      0.46      4898

RMSE:
0.8838185092014216
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1142   298   200
    2  847   556   795
    3  145   196   719

             precision    recall  f1-score   support

          1       0.54      0.70      0.61      1640
          2       0.53      0.25      0.34      2198
          3       0.42      0.68      0.52      1060

avg / total       0.51      0.49      0.47      4898

RMSE:
0.8472567603545674
#____________________white log normed____________________#


      __Prediction___
         1     2     3
    1 1088   263   289
    2  786   516   896
    3  138   166   756

             precision    recall  f1-score   support

          1       0.54      0.66      0.60      1640
          2       0.55      0.23      0.33      2198
          3       0.39      0.71      0.50      1060

avg / total       0.51      0.48      0.46      4898

RMSE:
0.8830096276088838
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1144   287   209
    2  849   534   815
    3  150   188   722

             precision    recall  f1-score   support

          1       0.53      0.70      0.60      1640
          2       0.53      0.24      0.33      2198
          3       0.41      0.68      0.51      1060

avg / total       0.51      0.49      0.46      4898

RMSE:
0.8543358537008395
#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1  935    76  

             precision    recall  f1-score   support

          1       0.53      0.70      0.61      1640
          2       0.53      0.24      0.33      2198
          3       0.41      0.69      0.51      1060

avg / total       0.51      0.49      0.46      4898

RMSE:
0.8567222713485284
#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1  938    62   640
    2 1011   198   989
    3  315   119   626

             precision    recall  f1-score   support

          1       0.41      0.57      0.48      1640
          2       0.52      0.09      0.15      2198
          3       0.28      0.59      0.38      1060

avg / total       0.43      0.36      0.31      4898

RMSE:
1.1068847983042442
1.3
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1089   264   287
    2  804   481   913
    3  140   160   760

             precision    recall  f1-score   support

        

      __Prediction___
         1     2     3
    1 1161   263   216
    2  859   511   828
    3  151   185   724

             precision    recall  f1-score   support

          1       0.53      0.71      0.61      1640
          2       0.53      0.23      0.32      2198
          3       0.41      0.68      0.51      1060

avg / total       0.51      0.49      0.46      4898

RMSE:
0.8576749792006976
#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1  946    48   646
    2 1025   176   997
    3  313   111   636

             precision    recall  f1-score   support

          1       0.41      0.58      0.48      1640
          2       0.53      0.08      0.14      2198
          3       0.28      0.60      0.38      1060

avg / total       0.43      0.36      0.31      4898

RMSE:
1.1083594165078847
1.9
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1104   242 

      __Prediction___
         1     2     3
    1 1164   257   219
    2  869   492   837
    3  157   175   728

             precision    recall  f1-score   support

          1       0.53      0.71      0.61      1640
          2       0.53      0.22      0.32      2198
          3       0.41      0.69      0.51      1060

avg / total       0.51      0.49      0.46      4898

RMSE:
0.86230435670551
#____________________white log normed____________________#
      __Prediction___
         1     2     3
    1  962    32   646
    2 1039   148  1011
    3  306   106   648

             precision    recall  f1-score   support

          1       0.42      0.59      0.49      1640
          2       0.52      0.07      0.12      2198
          3       0.28      0.61      0.39      1060

avg / total       0.43      0.36      0.30      4898

RMSE:
1.1064235770673758
2.5
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1114   223   

...

      __Prediction___
         1     2     3
    1 1182   150   308
    2  981   242   975
    3  184    78   798

             precision    recall  f1-score   support

          1       0.50      0.72      0.59      1640
          2       0.51      0.11      0.18      2198
          3       0.38      0.75      0.51      1060

avg / total       0.48      0.45      0.39      4898

RMSE:
0.9207024144055455
6.9
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1218   131   291
    2 1007   239   952
    3  184    93   783

             precision    recall  f1-score   support

          1       0.51      0.74      0.60      1640
          2       0.52      0.11      0.18      2198
          3       0.39      0.74      0.51      1060

avg / total       0.48      0.46      0.39      4898

RMSE:
0.9130200180100412
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1204   1

             precision    recall  f1-score   support

          1       0.51      0.72      0.60      1640
          2       0.53      0.13      0.20      2198
          3       0.39      0.75      0.51      1060

avg / total       0.49      0.46      0.40      4898

RMSE:
0.9102205387458344
7.6
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1225   122   293
    2 1044   195   959
    3  198    72   790

             precision    recall  f1-score   support

          1       0.50      0.75      0.60      1640
          2       0.50      0.09      0.15      2198
          3       0.39      0.75      0.51      1060

avg / total       0.47      0.45      0.38      4898

RMSE:
0.9216997453508536
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1208   176   256
    2  976   313   909
    3  188   103   769

             precision    recall  f1-score   support

     

             precision    recall  f1-score   support

          1       0.50      0.74      0.60      1640
          2       0.54      0.11      0.18      2198
          3       0.39      0.75      0.52      1060

avg / total       0.49      0.46      0.39      4898

RMSE:
0.9154764784108214
8.3
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1234   111   295
    2 1063   170   965
    3  203    62   795

             precision    recall  f1-score   support

          1       0.49      0.75      0.60      1640
          2       0.50      0.08      0.13      2198
          3       0.39      0.75      0.51      1060

avg / total       0.47      0.45      0.37      4898

RMSE:
0.9252371044598088
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1214   166   260
    2 1009   272   917
    3  197    92   771

             precision    recall  f1-score   support

     

             precision    recall  f1-score   support

          1       0.50      0.74      0.60      1640
          2       0.54      0.11      0.18      2198
          3       0.39      0.75      0.52      1060

avg / total       0.50      0.46      0.39      4898

RMSE:
0.9161452790443395
8.9
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1237   112   291
    2 1065   165   968
    3  206    60   794

             precision    recall  f1-score   support

          1       0.49      0.75      0.60      1640
          2       0.49      0.08      0.13      2198
          3       0.39      0.75      0.51      1060

avg / total       0.47      0.45      0.37      4898

RMSE:
0.9252371044598088
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1214   166   260
    2 1009   272   917
    3  197    92   771

             precision    recall  f1-score   support

     

      __Prediction___
         1     2     3
    1 1214   119   307
    2 1009   210   979
    3  197    60   803

             precision    recall  f1-score   support

          1       0.50      0.74      0.60      1640
          2       0.54      0.10      0.16      2198
          3       0.38      0.76      0.51      1060

avg / total       0.49      0.45      0.38      4898

RMSE:
0.9241331342486598
9.6
#____________________white STD normed____________________#
      __Prediction___
         1     2     3
    1 1259    88   293
    2 1092   150   956
    3  222    39   799

             precision    recall  f1-score   support

          1       0.49      0.77      0.60      1640
          2       0.54      0.07      0.12      2198
          3       0.39      0.75      0.51      1060

avg / total       0.49      0.45      0.37      4898

RMSE:
0.9298594668074489
#____________________white MinMax normed____________________#
      __Prediction___
         1     2     3
    1 1214   1

In [38]:
for t in range (0,10000,100):
    t = t/1000
    print(t)
    run_kncs_report(white_norm_input.values,white_targetclass,t)
    run_kncs_report(red_norm_input.values,red_targetclass,t)    

0.0
      __Prediction___
         1     2     3
    1 1078   272   290
    2  783   537   878
    3  138   181   741

             precision    recall  f1-score   support

          1       0.54      0.66      0.59      1640
          2       0.54      0.24      0.34      2198
          3       0.39      0.70      0.50      1060

avg / total       0.51      0.48      0.46      4898

RMSE:
0.8838185092014216
      __Prediction___
         1     2     3
    1  541   139    64
    2  219   223   196
    3   11    42   164

             precision    recall  f1-score   support

          1       0.70      0.73      0.71       744
          2       0.55      0.35      0.43       638
          3       0.39      0.76      0.51       217

avg / total       0.60      0.58      0.57      1599

RMSE:
0.7485654406174548
0.1
      __Prediction___
         1     2     3
    1 1080   270   290
    2  781   542   875
    3  138   180   742

             precision    recall  f1-score   support

       

             precision    recall  f1-score   support

          1       0.54      0.66      0.59      1640
          2       0.54      0.23      0.32      2198
          3       0.39      0.72      0.50      1060

avg / total       0.51      0.48      0.45      4898

RMSE:
0.8860103224836438
      __Prediction___
         1     2     3
    1  548   126    70
    2  213   187   238
    3   11    34   172

             precision    recall  f1-score   support

          1       0.71      0.74      0.72       744
          2       0.54      0.29      0.38       638
          3       0.36      0.79      0.49       217

avg / total       0.59      0.57      0.55      1599

RMSE:
0.7646832434343277
1.2
      __Prediction___
         1     2     3
    1 1092   260   288
    2  797   493   908
    3  140   159   761

             precision    recall  f1-score   support

          1       0.54      0.67      0.60      1640
          2       0.54      0.22      0.32      2198
          3       0.

      __Prediction___
         1     2     3
    1 1111   229   300
    2  827   454   917
    3  146   157   757

             precision    recall  f1-score   support

          1       0.53      0.68      0.60      1640
          2       0.54      0.21      0.30      2198
          3       0.38      0.71      0.50      1060

avg / total       0.50      0.47      0.44      4898

RMSE:
0.893924870530357
      __Prediction___
         1     2     3
    1  554   106    84
    2  216   163   259
    3   11    32   174

             precision    recall  f1-score   support

          1       0.71      0.74      0.73       744
          2       0.54      0.26      0.35       638
          3       0.34      0.80      0.47       217

avg / total       0.59      0.56      0.54      1599

RMSE:
0.7880438650279914
2.3
      __Prediction___
         1     2     3
    1 1107   229   304
    2  829   449   920
    3  146   157   757

             precision    recall  f1-score   support

          1 

      __Prediction___
         1     2     3
    1 1131   216   293
    2  869   407   922
    3  146   148   766

             precision    recall  f1-score   support

          1       0.53      0.69      0.60      1640
          2       0.53      0.19      0.27      2198
          3       0.39      0.72      0.50      1060

avg / total       0.50      0.47      0.43      4898

RMSE:
0.8935822174018682
      __Prediction___
         1     2     3
    1  557    92    95
    2  224   143   271
    3   11    23   183

             precision    recall  f1-score   support

          1       0.70      0.75      0.73       744
          2       0.55      0.22      0.32       638
          3       0.33      0.84      0.48       217

avg / total       0.59      0.55      0.53      1599

RMSE:
0.8041480950977132
3.5
      __Prediction___
         1     2     3
    1 1131   217   292
    2  872   406   920
    3  146   148   766

             precision    recall  f1-score   support

          1

      __Prediction___
         1     2     3
    1  574    58   112
    2  258    89   291
    3   16    22   179

             precision    recall  f1-score   support

          1       0.68      0.77      0.72       744
          2       0.53      0.14      0.22       638
          3       0.31      0.82      0.45       217

avg / total       0.57      0.53      0.48      1599

RMSE:
0.8447313074958598
4.7
      __Prediction___
         1     2     3
    1 1166   181   293
    2  917   351   930
    3  155   126   779

             precision    recall  f1-score   support

          1       0.52      0.71      0.60      1640
          2       0.53      0.16      0.25      2198
          3       0.39      0.73      0.51      1060

avg / total       0.50      0.47      0.42      4898

RMSE:
0.8975716979952398
      __Prediction___
         1     2     3
    1  573    56   115
    2  258    88   292
    3   16    22   179

             precision    recall  f1-score   support

          1

...

C:\Users\D060379\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\D060379\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\D060379\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\D060379\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined a

             precision    recall  f1-score   support

          1       0.47      1.00      0.64       744
          2       0.00      0.00      0.00       638
          3       0.00      0.00      0.00       217

avg / total       0.22      0.47      0.30      1599

RMSE:
0.970483719160565
9.6
      __Prediction___
         1     2     3
    1 1259    88   293
    2 1092   150   956
    3  222    39   799

             precision    recall  f1-score   support

          1       0.49      0.77      0.60      1640
          2       0.54      0.07      0.12      2198
          3       0.39      0.75      0.51      1060

avg / total       0.49      0.45      0.37      4898

RMSE:
0.9298594668074489
      __Prediction___
         1     2     3
    1  744     0     0
    2  638     0     0
    3  217     0     0

             precision    recall  f1-score   support

          1       0.47      1.00      0.64       744
          2       0.00      0.00      0.00       638
          3       0.0

C:\Users\D060379\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\D060379\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\D060379\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


      __Prediction___
         1     2     3
    1 1259    91   290
    2 1092   151   955
    3  222    38   800

             precision    recall  f1-score   support

          1       0.49      0.77      0.60      1640
          2       0.54      0.07      0.12      2198
          3       0.39      0.75      0.52      1060

avg / total       0.49      0.45      0.37      4898

RMSE:
0.9286510719281068
      __Prediction___
         1     2     3
    1  744     0     0
    2  638     0     0
    3  217     0     0

             precision    recall  f1-score   support

          1       0.47      1.00      0.64       744
          2       0.00      0.00      0.00       638
          3       0.00      0.00      0.00       217

avg / total       0.22      0.47      0.30      1599

RMSE:
0.970483719160565


C:\Users\D060379\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [39]:
for t in range (0,10000,100):
    t = t/1000
    print(t)
    print("#____________________red STD normed____________________#")
    run_kncs_report(red_norm.values, red_targetclass)
    print("#____________________red MinMax normed____________________#")
    run_kncs_report(mm_red_norm.values, red_targetclass)
    print("#____________________red log normed____________________#")
    run_kncs_report(log_red_norm.values, red_targetclass)

0.0
#____________________red STD normed____________________#
      __Prediction___
         1     2     3
    1  541   139    64
    2  219   223   196
    3   11    42   164

             precision    recall  f1-score   support

          1       0.70      0.73      0.71       744
          2       0.55      0.35      0.43       638
          3       0.39      0.76      0.51       217

avg / total       0.60      0.58      0.57      1599

RMSE:
0.7485654406174548
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  561   109    74
    2  241   200   197
    3   11    44   162

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.57      0.31      0.40       638
          3       0.37      0.75      0.50       217

avg / total       0.60      0.58      0.56      1599

RMSE:
0.763045804203175
#____________________red log normed____________________#
      _

             precision    recall  f1-score   support

          1       0.55      0.57      0.56       744
          2       0.47      0.15      0.23       638
          3       0.21      0.58      0.30       217

avg / total       0.47      0.41      0.39      1599

RMSE:
1.0747543605795422
0.7
#____________________red STD normed____________________#
      __Prediction___
         1     2     3
    1  541   139    64
    2  219   223   196
    3   11    42   164

             precision    recall  f1-score   support

          1       0.70      0.73      0.71       744
          2       0.55      0.35      0.43       638
          3       0.39      0.76      0.51       217

avg / total       0.60      0.58      0.57      1599

RMSE:
0.7485654406174548
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  561   109    74
    2  241   200   197
    3   11    44   162

             precision    recall  f1-score   support

         

      __Prediction___
         1     2     3
    1  541   139    64
    2  219   223   196
    3   11    42   164

             precision    recall  f1-score   support

          1       0.70      0.73      0.71       744
          2       0.55      0.35      0.43       638
          3       0.39      0.76      0.51       217

avg / total       0.60      0.58      0.57      1599

RMSE:
0.7485654406174548
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  561   109    74
    2  241   200   197
    3   11    44   162

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.57      0.31      0.40       638
          3       0.37      0.75      0.50       217

avg / total       0.60      0.58      0.56      1599

RMSE:
0.763045804203175
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  427    84   233


      __Prediction___
         1     2     3
    1  541   139    64
    2  219   223   196
    3   11    42   164

             precision    recall  f1-score   support

          1       0.70      0.73      0.71       744
          2       0.55      0.35      0.43       638
          3       0.39      0.76      0.51       217

avg / total       0.60      0.58      0.57      1599

RMSE:
0.7485654406174548
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  561   109    74
    2  241   200   197
    3   11    44   162

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.57      0.31      0.40       638
          3       0.37      0.75      0.50       217

avg / total       0.60      0.58      0.56      1599

RMSE:
0.763045804203175
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  427    84   233


             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.57      0.31      0.40       638
          3       0.37      0.75      0.50       217

avg / total       0.60      0.58      0.56      1599

RMSE:
0.763045804203175
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  427    84   233
    2  290    96   252
    3   66    25   126

             precision    recall  f1-score   support

          1       0.55      0.57      0.56       744
          2       0.47      0.15      0.23       638
          3       0.21      0.58      0.30       217

avg / total       0.47      0.41      0.39      1599

RMSE:
1.0747543605795422
2.7
#____________________red STD normed____________________#
      __Prediction___
         1     2     3
    1  541   139    64
    2  219   223   196
    3   11    42   164

             precision    recall  f1-score   support

          1  

...

             precision    recall  f1-score   support

          1       0.70      0.73      0.71       744
          2       0.55      0.35      0.43       638
          3       0.39      0.76      0.51       217

avg / total       0.60      0.58      0.57      1599

RMSE:
0.7485654406174548
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  561   109    74
    2  241   200   197
    3   11    44   162

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.57      0.31      0.40       638
          3       0.37      0.75      0.50       217

avg / total       0.60      0.58      0.56      1599

RMSE:
0.763045804203175
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  427    84   233
    2  290    96   252
    3   66    25   126

             precision    recall  f1-score   support

          1   

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.57      0.31      0.40       638
          3       0.37      0.75      0.50       217

avg / total       0.60      0.58      0.56      1599

RMSE:
0.763045804203175
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  427    84   233
    2  290    96   252
    3   66    25   126

             precision    recall  f1-score   support

          1       0.55      0.57      0.56       744
          2       0.47      0.15      0.23       638
          3       0.21      0.58      0.30       217

avg / total       0.47      0.41      0.39      1599

RMSE:
1.0747543605795422
7.6
#____________________red STD normed____________________#
      __Prediction___
         1     2     3
    1  541   139    64
    2  219   223   196
    3   11    42   164

             precision    recall  f1-score   support

          1  

             precision    recall  f1-score   support

          1       0.70      0.73      0.71       744
          2       0.55      0.35      0.43       638
          3       0.39      0.76      0.51       217

avg / total       0.60      0.58      0.57      1599

RMSE:
0.7485654406174548
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  561   109    74
    2  241   200   197
    3   11    44   162

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.57      0.31      0.40       638
          3       0.37      0.75      0.50       217

avg / total       0.60      0.58      0.56      1599

RMSE:
0.763045804203175
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  427    84   233
    2  290    96   252
    3   66    25   126

             precision    recall  f1-score   support

          1   

             precision    recall  f1-score   support

          1       0.70      0.73      0.71       744
          2       0.55      0.35      0.43       638
          3       0.39      0.76      0.51       217

avg / total       0.60      0.58      0.57      1599

RMSE:
0.7485654406174548
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  561   109    74
    2  241   200   197
    3   11    44   162

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.57      0.31      0.40       638
          3       0.37      0.75      0.50       217

avg / total       0.60      0.58      0.56      1599

RMSE:
0.763045804203175
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  427    84   233
    2  290    96   252
    3   66    25   126

             precision    recall  f1-score   support

          1   

             precision    recall  f1-score   support

          1       0.70      0.73      0.71       744
          2       0.55      0.35      0.43       638
          3       0.39      0.76      0.51       217

avg / total       0.60      0.58      0.57      1599

RMSE:
0.7485654406174548
#____________________red MinMax normed____________________#
      __Prediction___
         1     2     3
    1  561   109    74
    2  241   200   197
    3   11    44   162

             precision    recall  f1-score   support

          1       0.69      0.75      0.72       744
          2       0.57      0.31      0.40       638
          3       0.37      0.75      0.50       217

avg / total       0.60      0.58      0.56      1599

RMSE:
0.763045804203175
#____________________red log normed____________________#
      __Prediction___
         1     2     3
    1  427    84   233
    2  290    96   252
    3   66    25   126

             precision    recall  f1-score   support

          1   

In [40]:
def run_knnw_report(inputs, targets):
    print("_______________________________________________________________________________________________")
    print("KNN with inverse distance as weight:")
    for n_neighbour in range(1,31):
        print(str(n_neighbour) + " neighbours:")
        knn_estimator = KNeighborsClassifier(n_neighbour,weights='distance')
        print(str(n_neighbour)+ " neighbours//")
        predicted = cross_val_predict(knn_estimator,inputs,targets,cv=cv)
        print(confusion_matrix_report(targets,predicted))
        print(classification_report(targets,predicted))
        try:
            print("RMSE:")
            print(root_mean_squared_error(targets,predicted))
        except(Error):
            pass

In [41]:
print("#____________________red STD normed____________________#")
run_knnw_report(f_red_norm.values, red_targetclass)

#____________________red STD normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1  576   155    13
    2  150   407    81
    3   17    70   130

             precision    recall  f1-score   support

          1       0.78      0.77      0.77       744
          2       0.64      0.64      0.64       638
          3       0.58      0.60      0.59       217

avg / total       0.70      0.70      0.70      1599

RMSE:
0.6001875879364265
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  576   155    13
    2  151   406    81
    3   17    70   130

             precision    recall  f1-score   support

          1       0.77      0.77      0.77       744
          2       0.64      0.64      0.64       638
          3       0.58      0.60      0.59       217

avg / total    

      __Prediction___
         1     2     3
    1  611   129     4
    2  154   441    43
    3   15    81   121

             precision    recall  f1-score   support

          1       0.78      0.82      0.80       744
          2       0.68      0.69      0.68       638
          3       0.72      0.56      0.63       217

avg / total       0.73      0.73      0.73      1599

RMSE:
0.5496033022723462
23 neighbours:
23 neighbours//
      __Prediction___
         1     2     3
    1  616   123     5
    2  151   447    40
    3   15    81   121

             precision    recall  f1-score   support

          1       0.79      0.83      0.81       744
          2       0.69      0.70      0.69       638
          3       0.73      0.56      0.63       217

avg / total       0.74      0.74      0.74      1599

RMSE:
0.5450327172879821
24 neighbours:
24 neighbours//
      __Prediction___
         1     2     3
    1  614   125     5
    2  153   443    42
    3   15    81   121

       

In [42]:
print("#____________________red MinMax normed____________________#")
run_knnw_report(f_mm_red.values, red_targetclass)

#____________________red MinMax normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1  577   152    15
    2  139   420    79
    3   16    68   133

             precision    recall  f1-score   support

          1       0.79      0.78      0.78       744
          2       0.66      0.66      0.66       638
          3       0.59      0.61      0.60       217

avg / total       0.71      0.71      0.71      1599

RMSE:
0.5928487737550271
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  577   152    15
    2  140   419    79
    3   16    68   133

             precision    recall  f1-score   support

          1       0.79      0.78      0.78       744
          2       0.66      0.66      0.66       638
          3       0.59      0.61      0.60       217

avg / total 

             precision    recall  f1-score   support

          1       0.80      0.81      0.81       744
          2       0.68      0.71      0.69       638
          3       0.72      0.57      0.64       217

avg / total       0.74      0.74      0.74      1599

RMSE:
0.5392650129772822
20 neighbours:
20 neighbours//
      __Prediction___
         1     2     3
    1  604   138     2
    2  139   450    49
    3   14    79   124

             precision    recall  f1-score   support

          1       0.80      0.81      0.80       744
          2       0.67      0.71      0.69       638
          3       0.71      0.57      0.63       217

avg / total       0.74      0.74      0.74      1599

RMSE:
0.5415794657283098
21 neighbours:
21 neighbours//
      __Prediction___
         1     2     3
    1  605   137     2
    2  139   451    48
    3   14    78   125

             precision    recall  f1-score   support

          1       0.80      0.81      0.81       744
          2    

In [43]:
print("#____________________red log normed____________________#")
run_knnw_report(f_log_red.values, red_targetclass)

#____________________red log normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1  563   162    19
    2  159   395    84
    3   18    77   122

             precision    recall  f1-score   support

          1       0.76      0.76      0.76       744
          2       0.62      0.62      0.62       638
          3       0.54      0.56      0.55       217

avg / total       0.68      0.68      0.68      1599

RMSE:
0.6276912040603917
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  563   162    19
    2  159   395    84
    3   18    77   122

             precision    recall  f1-score   support

          1       0.76      0.76      0.76       744
          2       0.62      0.62      0.62       638
          3       0.54      0.56      0.55       217

avg / total    

             precision    recall  f1-score   support

          1       0.78      0.82      0.80       744
          2       0.66      0.69      0.67       638
          3       0.72      0.49      0.58       217

avg / total       0.72      0.72      0.72      1599

RMSE:
0.5575116698677923
23 neighbours:
23 neighbours//
      __Prediction___
         1     2     3
    1  613   126     5
    2  163   436    39
    3   15    95   107

             precision    recall  f1-score   support

          1       0.77      0.82      0.80       744
          2       0.66      0.68      0.67       638
          3       0.71      0.49      0.58       217

avg / total       0.72      0.72      0.72      1599

RMSE:
0.5608668355809016
24 neighbours:
24 neighbours//
      __Prediction___
         1     2     3
    1  608   131     5
    2  160   443    35
    3   16    96   105

             precision    recall  f1-score   support

          1       0.78      0.82      0.80       744
          2    

In [44]:
print("#____________________white STD normed____________________#")
run_knnw_report(white_norm.values, white_targetclass)

#____________________white STD normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1 1153   420    67
    2  372  1543   283
    3   57   277   726

             precision    recall  f1-score   support

          1       0.73      0.70      0.72      1640
          2       0.69      0.70      0.70      2198
          3       0.67      0.68      0.68      1060

avg / total       0.70      0.70      0.70      4898

RMSE:
0.6142449477688314
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1 1153   420    67
    2  372  1543   283
    3   57   277   726

             precision    recall  f1-score   support

          1       0.73      0.70      0.72      1640
          2       0.69      0.70      0.70      2198
          3       0.67      0.68      0.68      1060

avg / total  

      __Prediction___
         1     2     3
    1 1177   419    44
    2  322  1666   210
    3   33   318   709

             precision    recall  f1-score   support

          1       0.77      0.72      0.74      1640
          2       0.69      0.76      0.72      2198
          3       0.74      0.67      0.70      1060

avg / total       0.73      0.73      0.73      4898

RMSE:
0.5674223737793694
21 neighbours:
21 neighbours//
      __Prediction___
         1     2     3
    1 1177   420    43
    2  323  1664   211
    3   30   319   711

             precision    recall  f1-score   support

          1       0.77      0.72      0.74      1640
          2       0.69      0.76      0.72      2198
          3       0.74      0.67      0.70      1060

avg / total       0.73      0.73      0.73      4898

RMSE:
0.5652593835416719
22 neighbours:
22 neighbours//
      __Prediction___
         1     2     3
    1 1180   413    47
    2  320  1668   210
    3   31   325   704

       

In [45]:
print("#____________________white MinMax normed____________________#")
run_knnw_report(mm_white_norm.values, white_targetclass)

#____________________white MinMax normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1 1153   412    75
    2  380  1537   281
    3   57   288   715

             precision    recall  f1-score   support

          1       0.73      0.70      0.71      1640
          2       0.69      0.70      0.69      2198
          3       0.67      0.67      0.67      1060

avg / total       0.70      0.70      0.70      4898

RMSE:
0.6210214323487594
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1 1153   412    75
    2  380  1537   281
    3   57   288   715

             precision    recall  f1-score   support

          1       0.73      0.70      0.71      1640
          2       0.69      0.70      0.69      2198
          3       0.67      0.67      0.67      1060

avg / tota

      __Prediction___
         1     2     3
    1 1143   433    64
    2  333  1632   233
    3   27   321   712

             precision    recall  f1-score   support

          1       0.76      0.70      0.73      1640
          2       0.68      0.74      0.71      2198
          3       0.71      0.67      0.69      1060

avg / total       0.71      0.71      0.71      4898

RMSE:
0.5863563775995395
21 neighbours:
21 neighbours//
      __Prediction___
         1     2     3
    1 1146   431    63
    2  335  1630   233
    3   24   326   710

             precision    recall  f1-score   support

          1       0.76      0.70      0.73      1640
          2       0.68      0.74      0.71      2198
          3       0.71      0.67      0.69      1060

avg / total       0.71      0.71      0.71      4898

RMSE:
0.5844381805917904
22 neighbours:
22 neighbours//
      __Prediction___
         1     2     3
    1 1151   426    63
    2  329  1638   231
    3   25   326   709

       

In [46]:
print("#____________________white log normed____________________#")
run_knnw_report(log_white_norm.values, white_targetclass)

#____________________white log normed____________________#
_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1 1151   402    87
    2  420  1485   293
    3   69   299   692

             precision    recall  f1-score   support

          1       0.70      0.70      0.70      1640
          2       0.68      0.68      0.68      2198
          3       0.65      0.65      0.65      1060

avg / total       0.68      0.68      0.68      4898

RMSE:
0.6450489898178324
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1 1151   402    87
    2  420  1485   293
    3   69   299   692

             precision    recall  f1-score   support

          1       0.70      0.70      0.70      1640
          2       0.68      0.68      0.68      2198
          3       0.65      0.65      0.65      1060

avg / total  

             precision    recall  f1-score   support

          1       0.75      0.68      0.71      1640
          2       0.66      0.74      0.70      2198
          3       0.70      0.62      0.66      1060

avg / total       0.70      0.69      0.69      4898

RMSE:
0.5982484301331669
20 neighbours:
20 neighbours//
      __Prediction___
         1     2     3
    1 1112   478    50
    2  335  1641   222
    3   34   382   644

             precision    recall  f1-score   support

          1       0.75      0.68      0.71      1640
          2       0.66      0.75      0.70      2198
          3       0.70      0.61      0.65      1060

avg / total       0.70      0.69      0.69      4898

RMSE:
0.5982484301331669
21 neighbours:
21 neighbours//
      __Prediction___
         1     2     3
    1 1110   479    51
    2  325  1647   226
    3   33   376   651

             precision    recall  f1-score   support

          1       0.76      0.68      0.71      1640
          2    

In [47]:
run_knnw_report(white_input.values,white_targetclass)

_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1 1044   451   145
    2  430  1436   332
    3  113   329   618

             precision    recall  f1-score   support

          1       0.66      0.64      0.65      1640
          2       0.65      0.65      0.65      2198
          3       0.56      0.58      0.57      1060

avg / total       0.63      0.63      0.63      4898

RMSE:
0.7249280106751942
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1 1044   451   145
    2  430  1436   332
    3  113   329   618

             precision    recall  f1-score   support

          1       0.66      0.64      0.65      1640
          2       0.65      0.65      0.65      2198
          3       0.56      0.58      0.57      1060

avg / total       0.63      0.63      0.63      4898

RMSE:
0.7249280106

      __Prediction___
         1     2     3
    1 1059   506    75
    2  388  1602   208
    3   83   411   566

             precision    recall  f1-score   support

          1       0.69      0.65      0.67      1640
          2       0.64      0.73      0.68      2198
          3       0.67      0.53      0.59      1060

avg / total       0.66      0.66      0.66      4898

RMSE:
0.6617657066902496
21 neighbours:
21 neighbours//
      __Prediction___
         1     2     3
    1 1058   511    71
    2  401  1600   197
    3   86   419   555

             precision    recall  f1-score   support

          1       0.68      0.65      0.66      1640
          2       0.63      0.73      0.68      2198
          3       0.67      0.52      0.59      1060

avg / total       0.66      0.66      0.65      4898

RMSE:
0.6634603719661462
22 neighbours:
22 neighbours//
      __Prediction___
         1     2     3
    1 1050   515    75
    2  391  1618   189
    3   80   423   557

       

In [48]:
run_knnw_report(red_input.values,red_targetclass)

_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1  520   191    33
    2  190   380    68
    3   30    74   113

             precision    recall  f1-score   support

          1       0.70      0.70      0.70       744
          2       0.59      0.60      0.59       638
          3       0.53      0.52      0.52       217

avg / total       0.63      0.63      0.63      1599

RMSE:
0.6961881381511136
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  520   191    33
    2  190   380    68
    3   30    74   113

             precision    recall  f1-score   support

          1       0.70      0.70      0.70       744
          2       0.59      0.60      0.59       638
          3       0.53      0.52      0.52       217

avg / total       0.63      0.63      0.63      1599

RMSE:
0.6961881381

      __Prediction___
         1     2     3
    1  579   158     7
    2  222   384    32
    3   40    87    90

             precision    recall  f1-score   support

          1       0.69      0.78      0.73       744
          2       0.61      0.60      0.61       638
          3       0.70      0.41      0.52       217

avg / total       0.66      0.66      0.65      1599

RMSE:
0.6554719881158179
21 neighbours:
21 neighbours//
      __Prediction___
         1     2     3
    1  580   156     8
    2  226   385    27
    3   42    86    89

             precision    recall  f1-score   support

          1       0.68      0.78      0.73       744
          2       0.61      0.60      0.61       638
          3       0.72      0.41      0.52       217

avg / total       0.66      0.66      0.65      1599

RMSE:
0.6592773727035149
22 neighbours:
22 neighbours//
      __Prediction___
         1     2     3
    1  574   163     7
    2  217   393    28
    3   42    86    89

       

In [49]:
run_knnw_report(red_norm.values,red_targetclass)

_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1  563   157    24
    2  162   404    72
    3   24    64   129

             precision    recall  f1-score   support

          1       0.75      0.76      0.75       744
          2       0.65      0.63      0.64       638
          3       0.57      0.59      0.58       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.6361036805684829
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  563   157    24
    2  162   404    72
    3   24    64   129

             precision    recall  f1-score   support

          1       0.75      0.76      0.75       744
          2       0.65      0.63      0.64       638
          3       0.57      0.59      0.58       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.6361036805

      __Prediction___
         1     2     3
    1  589   143    12
    2  144   443    51
    3   14    78   125

             precision    recall  f1-score   support

          1       0.79      0.79      0.79       744
          2       0.67      0.69      0.68       638
          3       0.66      0.58      0.62       217

avg / total       0.72      0.72      0.72      1599

RMSE:
0.570265948512201
21 neighbours:
21 neighbours//
      __Prediction___
         1     2     3
    1  593   139    12
    2  142   443    53
    3   10    86   121

             precision    recall  f1-score   support

          1       0.80      0.80      0.80       744
          2       0.66      0.69      0.68       638
          3       0.65      0.56      0.60       217

avg / total       0.72      0.72      0.72      1599

RMSE:
0.5636475508693359
22 neighbours:
22 neighbours//
      __Prediction___
         1     2     3
    1  593   139    12
    2  141   445    52
    3   10    81   126

        

In [50]:
run_knnw_report(mm_red_norm.values,red_targetclass)

_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1  555   166    23
    2  161   414    63
    3   24    64   129

             precision    recall  f1-score   support

          1       0.75      0.75      0.75       744
          2       0.64      0.65      0.65       638
          3       0.60      0.59      0.60       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.6336410167329005
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  555   166    23
    2  161   414    63
    3   24    64   129

             precision    recall  f1-score   support

          1       0.75      0.75      0.75       744
          2       0.64      0.65      0.65       638
          3       0.60      0.59      0.60       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.6336410167

             precision    recall  f1-score   support

          1       0.79      0.79      0.79       744
          2       0.66      0.69      0.67       638
          3       0.66      0.56      0.61       217

avg / total       0.72      0.72      0.72      1599

RMSE:
0.5669664071325218
21 neighbours:
21 neighbours//
      __Prediction___
         1     2     3
    1  601   132    11
    2  145   439    54
    3   10    83   124

             precision    recall  f1-score   support

          1       0.79      0.81      0.80       744
          2       0.67      0.69      0.68       638
          3       0.66      0.57      0.61       217

avg / total       0.73      0.73      0.73      1599

RMSE:
0.5580722649514734
22 neighbours:
22 neighbours//
      __Prediction___
         1     2     3
    1  595   137    12
    2  148   441    49
    3   11    82   124

             precision    recall  f1-score   support

          1       0.79      0.80      0.79       744
          2    

In [51]:
run_knnw_report(log_red_norm.values,red_targetclass)

_______________________________________________________________________________________________
KNN with inverse distance as weight:
1 neighbours:
1 neighbours//
      __Prediction___
         1     2     3
    1  555   162    27
    2  171   392    75
    3   21    67   129

             precision    recall  f1-score   support

          1       0.74      0.75      0.74       744
          2       0.63      0.61      0.62       638
          3       0.56      0.59      0.58       217

avg / total       0.67      0.67      0.67      1599

RMSE:
0.6458604414412116
2 neighbours:
2 neighbours//
      __Prediction___
         1     2     3
    1  555   162    27
    2  171   392    75
    3   21    67   129

             precision    recall  f1-score   support

          1       0.74      0.75      0.74       744
          2       0.63      0.61      0.62       638
          3       0.56      0.59      0.58       217

avg / total       0.67      0.67      0.67      1599

RMSE:
0.6458604414

             precision    recall  f1-score   support

          1       0.75      0.78      0.76       744
          2       0.63      0.65      0.64       638
          3       0.66      0.48      0.56       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.6027869400206114
22 neighbours:
22 neighbours//
      __Prediction___
         1     2     3
    1  587   151     6
    2  175   418    45
    3   21    93   103

             precision    recall  f1-score   support

          1       0.75      0.79      0.77       744
          2       0.63      0.66      0.64       638
          3       0.67      0.47      0.56       217

avg / total       0.69      0.69      0.69      1599

RMSE:
0.5980999726097406
23 neighbours:
23 neighbours//
      __Prediction___
         1     2     3
    1  585   154     5
    2  172   420    46
    3   25    90   102

             precision    recall  f1-score   support

          1       0.75      0.79      0.77       744
          2    

In [52]:
run_knn_report(f_mm_white.values,white_targetclass)
run_knnw_report(f_mm_white.values,white_targetclass)

_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1131   414    95
    2  441  1419   338
    3   82   325   653

             precision    recall  f1-score   support

          1       0.68      0.69      0.69      1640
          2       0.66      0.65      0.65      2198
          3       0.60      0.62      0.61      1060

avg / total       0.65      0.65      0.65      4898

RMSE:
0.6741448010182188
2 neighbours:
      __Prediction___
         1     2     3
    1 1285   319    36
    2  796  1242   160
    3  183   503   374

             precision    recall  f1-score   support

          1       0.57      0.78      0.66      1640
          2       0.60      0.57      0.58      2198
          3       0.66      0.35      0.46      1060

avg / total       0.60      0.59      0.58      4898

RMSE:
0.7361072054292438
3 neighbours:
      __Prediction___
        

      __Prediction___
         1     2     3
    1 1118   430    92
    2  436  1435   327
    3   75   334   651

             precision    recall  f1-score   support

          1       0.69      0.68      0.68      1640
          2       0.65      0.65      0.65      2198
          3       0.61      0.61      0.61      1060

avg / total       0.65      0.65      0.65      4898

RMSE:
0.6694341631675539
4 neighbours:
4 neighbours//
      __Prediction___
         1     2     3
    1 1128   428    84
    2  421  1458   319
    3   67   347   646

             precision    recall  f1-score   support

          1       0.70      0.69      0.69      1640
          2       0.65      0.66      0.66      2198
          3       0.62      0.61      0.61      1060

avg / total       0.66      0.66      0.66      4898

RMSE:
0.6577427775762
5 neighbours:
5 neighbours//
      __Prediction___
         1     2     3
    1 1132   433    75
    2  397  1489   312
    3   63   349   648

             p

      __Prediction___
         1     2     3
    1 1142   449    49
    2  352  1600   246
    3   43   370   647

             precision    recall  f1-score   support

          1       0.74      0.70      0.72      1640
          2       0.66      0.73      0.69      2198
          3       0.69      0.61      0.65      1060

avg / total       0.69      0.69      0.69      4898

RMSE:
0.6036840755280375
24 neighbours:
24 neighbours//
      __Prediction___
         1     2     3
    1 1150   440    50
    2  347  1600   251
    3   42   376   642

             precision    recall  f1-score   support

          1       0.75      0.70      0.72      1640
          2       0.66      0.73      0.69      2198
          3       0.68      0.61      0.64      1060

avg / total       0.69      0.69      0.69      4898

RMSE:
0.6031765646560429
25 neighbours:
25 neighbours//
      __Prediction___
         1     2     3
    1 1146   445    49
    2  338  1611   249
    3   42   370   648

       

In [53]:
run_knn_report(f_mm_white.values,white_targetclass)
run_knnw_report(f_mm_white.values,white_targetclass)

_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1131   414    95
    2  441  1419   338
    3   82   325   653

             precision    recall  f1-score   support

          1       0.68      0.69      0.69      1640
          2       0.66      0.65      0.65      2198
          3       0.60      0.62      0.61      1060

avg / total       0.65      0.65      0.65      4898

RMSE:
0.6741448010182188
2 neighbours:
      __Prediction___
         1     2     3
    1 1285   319    36
    2  796  1242   160
    3  183   503   374

             precision    recall  f1-score   support

          1       0.57      0.78      0.66      1640
          2       0.60      0.57      0.58      2198
          3       0.66      0.35      0.46      1060

avg / total       0.60      0.59      0.58      4898

RMSE:
0.7361072054292438
3 neighbours:
      __Prediction___
        

      __Prediction___
         1     2     3
    1 1118   430    92
    2  436  1435   327
    3   75   334   651

             precision    recall  f1-score   support

          1       0.69      0.68      0.68      1640
          2       0.65      0.65      0.65      2198
          3       0.61      0.61      0.61      1060

avg / total       0.65      0.65      0.65      4898

RMSE:
0.6694341631675539
4 neighbours:
4 neighbours//
      __Prediction___
         1     2     3
    1 1128   428    84
    2  421  1458   319
    3   67   347   646

             precision    recall  f1-score   support

          1       0.70      0.69      0.69      1640
          2       0.65      0.66      0.66      2198
          3       0.62      0.61      0.61      1060

avg / total       0.66      0.66      0.66      4898

RMSE:
0.6577427775762
5 neighbours:
5 neighbours//
      __Prediction___
         1     2     3
    1 1132   433    75
    2  397  1489   312
    3   63   349   648

             p

      __Prediction___
         1     2     3
    1 1146   445    49
    2  353  1588   257
    3   43   372   645

             precision    recall  f1-score   support

          1       0.74      0.70      0.72      1640
          2       0.66      0.72      0.69      2198
          3       0.68      0.61      0.64      1060

avg / total       0.69      0.69      0.69      4898

RMSE:
0.6053727056112299
23 neighbours:
23 neighbours//
      __Prediction___
         1     2     3
    1 1142   449    49
    2  352  1600   246
    3   43   370   647

             precision    recall  f1-score   support

          1       0.74      0.70      0.72      1640
          2       0.66      0.73      0.69      2198
          3       0.69      0.61      0.65      1060

avg / total       0.69      0.69      0.69      4898

RMSE:
0.6036840755280375
24 neighbours:
24 neighbours//
      __Prediction___
         1     2     3
    1 1150   440    50
    2  347  1600   251
    3   42   376   642

       

In [54]:
run_knn_report(f_log_white.values,white_targetclass)
run_knnw_report(f_log_white.values,white_targetclass)

_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1 1090   455    95
    2  459  1372   367
    3  102   330   628

             precision    recall  f1-score   support

          1       0.66      0.66      0.66      1640
          2       0.64      0.62      0.63      2198
          3       0.58      0.59      0.58      1060

avg / total       0.63      0.63      0.63      4898

RMSE:
0.6998512354317896
2 neighbours:
      __Prediction___
         1     2     3
    1 1278   325    37
    2  807  1229   162
    3  203   507   350

             precision    recall  f1-score   support

          1       0.56      0.78      0.65      1640
          2       0.60      0.56      0.58      2198
          3       0.64      0.33      0.44      1060

avg / total       0.59      0.58      0.57      4898

RMSE:
0.750799220278691
3 neighbours:
      __Prediction___
         

      __Prediction___
         1     2     3
    1 1101   445    94
    2  447  1418   333
    3   82   357   621

             precision    recall  f1-score   support

          1       0.68      0.67      0.67      1640
          2       0.64      0.65      0.64      2198
          3       0.59      0.59      0.59      1060

avg / total       0.64      0.64      0.64      4898

RMSE:
0.6831698988226949
4 neighbours:
4 neighbours//
      __Prediction___
         1     2     3
    1 1115   447    78
    2  432  1431   335
    3   75   359   626

             precision    recall  f1-score   support

          1       0.69      0.68      0.68      1640
          2       0.64      0.65      0.65      2198
          3       0.60      0.59      0.60      1060

avg / total       0.65      0.65      0.65      4898

RMSE:
0.6679075154262929
5 neighbours:
5 neighbours//
      __Prediction___
         1     2     3
    1 1105   457    78
    2  413  1470   315
    3   70   361   629

           

      __Prediction___
         1     2     3
    1 1149   451    40
    2  353  1617   228
    3   41   402   617

             precision    recall  f1-score   support

          1       0.74      0.70      0.72      1640
          2       0.65      0.74      0.69      2198
          3       0.70      0.58      0.63      1060

avg / total       0.69      0.69      0.69      4898

RMSE:
0.5991010006528603
24 neighbours:
24 neighbours//
      __Prediction___
         1     2     3
    1 1156   446    38
    2  345  1623   230
    3   41   407   612

             precision    recall  f1-score   support

          1       0.75      0.70      0.73      1640
          2       0.66      0.74      0.69      2198
          3       0.70      0.58      0.63      1060

avg / total       0.70      0.69      0.69      4898

RMSE:
0.5967107334958632
25 neighbours:
25 neighbours//
      __Prediction___
         1     2     3
    1 1161   443    36
    2  343  1633   222
    3   37   409   614

       

In [55]:
run_knn_report(f_mm_red.values,red_targetclass)
run_knnw_report(f_mm_red.values,red_targetclass)

_______________________________________________________________________________________________
KNN, no weights
1 neighbours:
      __Prediction___
         1     2     3
    1  577   152    15
    2  139   420    79
    3   16    68   133

             precision    recall  f1-score   support

          1       0.79      0.78      0.78       744
          2       0.66      0.66      0.66       638
          3       0.59      0.61      0.60       217

avg / total       0.71      0.71      0.71      1599

RMSE:
0.5928487737550271
2 neighbours:
      __Prediction___
         1     2     3
    1  641    99     4
    2  268   334    36
    3   37   122    58

             precision    recall  f1-score   support

          1       0.68      0.86      0.76       744
          2       0.60      0.52      0.56       638
          3       0.59      0.27      0.37       217

avg / total       0.64      0.65      0.63      1599

RMSE:
0.6564254024206326
3 neighbours:
      __Prediction___
        

      __Prediction___
         1     2     3
    1  583   148    13
    2  133   427    78
    3   17    69   131

             precision    recall  f1-score   support

          1       0.80      0.78      0.79       744
          2       0.66      0.67      0.67       638
          3       0.59      0.60      0.60       217

avg / total       0.71      0.71      0.71      1599

RMSE:
0.5854179672445431
4 neighbours:
4 neighbours//
      __Prediction___
         1     2     3
    1  584   146    14
    2  132   433    73
    3   19    71   127

             precision    recall  f1-score   support

          1       0.79      0.78      0.79       744
          2       0.67      0.68      0.67       638
          3       0.59      0.59      0.59       217

avg / total       0.72      0.72      0.72      1599

RMSE:
0.5886140854486008
5 neighbours:
5 neighbours//
      __Prediction___
         1     2     3
    1  597   136    11
    2  132   441    65
    3   17    76   124

           

      __Prediction___
         1     2     3
    1  612   129     3
    2  142   448    48
    3   13    86   118

             precision    recall  f1-score   support

          1       0.80      0.82      0.81       744
          2       0.68      0.70      0.69       638
          3       0.70      0.54      0.61       217

avg / total       0.74      0.74      0.73      1599

RMSE:
0.5415794657283098
26 neighbours:
26 neighbours//
      __Prediction___
         1     2     3
    1  610   130     4
    2  145   445    48
    3   13    84   120

             precision    recall  f1-score   support

          1       0.79      0.82      0.81       744
          2       0.68      0.70      0.69       638
          3       0.70      0.55      0.62       217

avg / total       0.73      0.73      0.73      1599

RMSE:
0.5450327172879821
27 neighbours:
27 neighbours//
      __Prediction___
         1     2     3
    1  609   131     4
    2  139   454    45
    3   11    88   118

       